In [1]:
import sys
sys.path.append("/home/benjamin/RewardCurriculum/")

import os
os.system("export MKL_SERVICE_FORCE_INTEL=1")

import gymnasium as gym
import panda_gym
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
import json
import numpy as np

import os
from copy import deepcopy
from utils.configs import get_config
from utils.env_wrappers import make_vec_env, get_env
from argparse import Namespace
import cv2
from IPython.display import HTML

/home/benjamin/miniconda3/envs/rew_curr/lib/python3.10/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


In [8]:
os.chdir("/home/benjamin/RewardCurriculum")
folder_path = "/home/benjamin/RewardCurriculum/results/panda_pick_and_place_obstacle_long/2024-05-21_17-06-37_PandaMultiRewardPickAndPlaceObstacleDense-v3_tqc_sacx_4"

args = Namespace()
args.env_name = json.load(open(os.path.join(folder_path, "config.json"), "r"))["environment"]["env_name"]
args.seed = json.load(open(os.path.join(folder_path, "config.json"), "r"))["seed"]
args.config_path = os.path.join(folder_path, "config_original.json")
args.continue_from = None

remaining_args = ["--environment.wrapper_kwargs.0.reward_threshold", "-0.05"]
# remaining_args.__setattr__[] = "-0.05"
config = get_config(args.config_path, args, remaining_args)
config["environment"]["wrappers"] += ["SingleTaskRewardWrapper"]
config["environment"]["wrapper_kwargs"] += [{}]

In [9]:
make_env_fn = lambda wrappers, wrapper_kwargs, ignore_keyword="ignore" : get_env(config["environment"]["env_name"], wrappers=wrappers, wrapper_kwargs=wrapper_kwargs, ignore_keyword=ignore_keyword)
env = make_vec_env(make_env_fn, 
                    n_envs=config["environment"]["n_envs"], 
                    env_kwargs={"wrappers": config["environment"]["wrappers"], "wrapper_kwargs": config["environment"]["wrapper_kwargs"]},
                    monitor_kwargs={"allow_early_resets": True},
                    seed=config["seed"], vec_env_cls=DummyVecEnv)

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


In [10]:
# learner = config["learner_class"].load(os.path.join(folder_path, "evaluations", "best_model.zip"), env=env)
learner = config["learner_class"].load(os.path.join(folder_path, "evaluations", "best_model.zip"), env=env)

In [13]:
def record_video(video_output_path, env, learner, task=None):
    if task is None:
        weights = np.zeros((1, learner.scheduler.reward_dim))
        weights[:, -1] = 1
    obs = env.reset()

    terminated = False
    render_imgs = []
    while not terminated:
        # obs = obs.reshape(1, -1)
        # print(obs)
        act = learner.predict(obs, weights=weights)[0]

        obs, reward, terminated, info = env.step(act)
        # print(reward)
        
        render_imgs.append(env.render())
        
    print(reward)
    height, width, _ = render_imgs[0].shape

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(video_output_path, fourcc=fourcc, fps=10, frameSize=(width, height))

    for frame in render_imgs:
        out.write(frame)
        
    out.release()
    
    os.system(f"ffmpeg -hide_banner -loglevel error -i {video_output_path} -vcodec libx264 video.mp4")
    os.system(f"mv video.mp4 {video_output_path}")

In [14]:
video_path = os.path.join(folder_path, "video.mp4")
record_video(video_path, env, learner)
HTML(f'<video alt="test" controls><source src="{video_path}" type="video/mp4"></video>')

[1.]


# Render environment

In [ ]:
import panda_gym
import gymnasium as gym

env = gym.make("PandaPickAndPlaceDense-v3", renderer="OpenGL")
obs = env.reset()
env.render()